# **Bayesian Meta-Learning**

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp

In [ ]:
def generate_synthetic_data(num_tasks, num_samples_per_task, input_dim, output_dim):
    data = []
    for _ in range(num_tasks):
        x = np.random.rand(num_samples_per_task, input_dim)
        y = np.random.randint(output_dim, size=num_samples_per_task)
        data.append((x, y))
    return data
tfd = tfp.distributions
class BayesianNN(tf.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.fc1 = tf.keras.layers.Dense(hidden_dim, activation='relu')
        self.fc2 = tf.keras.layers.Dense(output_dim)

    def __call__(self, x, samples=1):
        outputs = []
        for _ in range(samples):
            h1 = self.fc1(x)
            out = self.fc2(h1)
            outputs.append(out)
        return outputs

# Bayesian meta-learning model
class BayesianMetaLearningModel(tf.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.meta_learner = BayesianNN(input_dim, hidden_dim, output_dim)
        # prior distribution for task-specific parameters
        self.prior = tfd.Normal(loc=0., scale=1.)

    def __call__(self, data, task_specific_params, samples=1):
        outputs = []
        for _ in range(samples):
            task_params = self.prior.sample(task_specific_params.shape)
            for i in range(len(data)):
                task_input = data[i][0]
                task_input = tf.convert_to_tensor(task_input, dtype=tf.float32)
                task_output = self.meta_learner(task_input)
                outputs.append(task_output)
        return outputs

# training
num_tasks = 10
num_samples_per_task = 5
input_dim = 2
output_dim = 3
hidden_dim = 16
num_epochs = 100
learning_rate = 0.001

# synthetic data generation
training_data = generate_synthetic_data(num_tasks, num_samples_per_task, input_dim, output_dim)

# model initialization
model = BayesianMetaLearningModel(input_dim, hidden_dim, output_dim)
optimizer = tf.optimizers.Adam(learning_rate)


for epoch in range(num_epochs):
    for task_data in training_data:
        with tf.GradientTape() as tape:
            task_specific_params = tf.random.normal(shape=(1,))
            task_input = task_data[0]
            task_input = tf.convert_to_tensor(task_input, dtype=tf.float32)
            task_predictions = model([task_data], task_specific_params)
            loss = tf.losses.sparse_categorical_crossentropy(task_data[1], task_predictions[-1][0])
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# meta-testing
meta_test_task_data = generate_synthetic_data(1, num_samples_per_task, input_dim, output_dim)
meta_test_task_params = tf.random.normal(shape=(1,))
meta_test_predictions = model(meta_test_task_data, meta_test_task_params)

# evaluation
bayesian_nn_model = BayesianNN(input_dim, hidden_dim, output_dim)
print(meta_test_predictions)

